### optuna 튜닝


In [114]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random
from imblearn.under_sampling import RandomUnderSampler
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
plt.style.use("ggplot")
import lightgbm as lgbm


In [3]:
train_df = pd.read_csv('../Data/multi-label-classification-data/train_process.csv')
test_df = pd.read_csv('../Data/multi-label-classification-data/test.csv')
label_df = pd.read_csv('../Data/multi-label-classification-data/label.csv')
submission = pd.read_csv('../Data/multi-label-classification-data/sample_submission.csv')

In [4]:
train_df = train_df.drop(['id','FpDensityMorgan1', 'FpDensityMorgan2','FpDensityMorgan3'], axis=1)
test_df = test_df.drop(['id','FpDensityMorgan1', 'FpDensityMorgan2','FpDensityMorgan3'], axis=1)


In [5]:
# skew_feature = ['BertzCT', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3v', 'Chi4n',
#        'EState_VSA1', 'EState_VSA2', 'ExactMolWt', 'FpDensityMorgan1',
#        'FpDensityMorgan2', 'FpDensityMorgan3',
#        'HeavyAtomMolWt', 'Kappa3', 
#        'NumHeteroatoms', 'PEOE_VSA10', 'PEOE_VSA14', 'PEOE_VSA6', 'PEOE_VSA7',
#        'PEOE_VSA8', 'SMR_VSA10', 'SMR_VSA5', 'SlogP_VSA3', 'VSA_EState9',
#        ]
skew_feature = [ 'Kappa3']

In [6]:
for col in skew_feature:
    train_df[col] = np.log1p(train_df[col])
    test_df[col] = np.log1p(test_df[col])


/home/bbok0525/anaconda3/envs/default/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [7]:
# 훈련 검증 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(train_df, label_df,test_size=0.3)

In [8]:
cols = label_df.columns

In [161]:
# optuna 사용
import optuna
from optuna import Trial
from sklearn.metrics import roc_curve

train = lgbm.Dataset(x_train, y_train.iloc[:,0])
test = lgbm.Dataset(x_test, y_test.iloc[:,0])
def objective(trial : Trial):
    params = {
        'boosting_type' : 'gbdt',
        "n_estimators" : 10000,
#         'fold_size':trial.suggest_int('fold_size', 4, 16),
        'max_depth':trial.suggest_int('max_depth', 4, 16),
        'random_state': 722,
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 8, 25),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.8, 1.0),
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 8),
        'min_child_samples': trial.suggest_int('min_child_samples', 16, 64),
        'learning_rate': 0.05,
        "metric":"auc"
    }
    
    model = lgbm.train(params=params, train_set=train, valid_sets=[train, test], num_boost_round=500, early_stopping_rounds=10, verbose_eval=100)
    preds = model.predict(x_test)
    print(preds)
    fpr, tpr, thresholds = roc_curve(y_test['EC1'], preds, pos_label=1)
    
    score = auc(fpr, tpr)
    
    return score
    

In [162]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
print("trial",len(study.trials))
print('best_trail', study.best_trial.params)

/home/bbok0525/anaconda3/envs/default/lib/python3.10/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2023-07-07 13:26:01,730] Trial 0 finished with value: 0.7151372239577389 and parameters: {'max_depth': 12, 'reg_alpha': 0.00420419296206051, 'reg_lambda': 0.15614653920958144, 'num_leaves': 15, 'colsample_bytree': 0.857093654788634, 'subsample': 0.8774383496487371, 'subsample_freq': 3, 'min_child_samples': 30}. Best is trial 0 with value: 0.7151372239577389.
/home/bbok0525/anaconda3/envs/default/lib/python3.10/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2023-07-07 13:26:01,770] Trial 1 finished with value: 0.7163176253447335 and parameters: {'max_depth': 11, 'reg_alpha': 3.7

[LightGBM] [Info] Total Bins 5562
[LightGBM] [Info] Number of data: 10386, number of used features: 27
[LightGBM] [Info] Start training from score 0.665030
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[46]	training's auc: 0.740454	valid_1's auc: 0.715137
[0.8137905  0.50669855 0.74420077 ... 0.50953958 0.77892863 0.46777087]
[LightGBM] [Info] Total Bins 5562
[LightGBM] [Info] Number of data: 10386, number of used features: 27
[LightGBM] [Info] Start training from score 0.665030
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[43]	training's auc: 0.723977	valid_1's auc: 0.716318
[0.80150425 0.48100845 0.75906508 ... 0.52729596 0.76238018 0.47676246]
[LightGBM] [Info] Total Bins 5562
[LightGBM] [Info] Number of data: 10386, number of used features: 27
[LightGBM] [Info] Start training from score 0.665030
Training until validation scores don't improve for 10 rounds
Early stopping, best iterat

[I 2023-07-07 13:26:01,895] Trial 3 finished with value: 0.7155674733737962 and parameters: {'max_depth': 4, 'reg_alpha': 0.07847714011798672, 'reg_lambda': 8.978909363827826, 'num_leaves': 15, 'colsample_bytree': 0.8503227686267543, 'subsample': 0.8630060761080658, 'subsample_freq': 2, 'min_child_samples': 39}. Best is trial 1 with value: 0.7163176253447335.
/home/bbok0525/anaconda3/envs/default/lib/python3.10/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2023-07-07 13:26:01,925] Trial 4 finished with value: 0.7129340759604368 and parameters: {'max_depth': 16, 'reg_alpha': 0.19568039103955048, 'reg_lambda': 0.008825125973778868, 'num_leaves': 19, 'colsample_bytree': 0.8082103101213838, 'subsample': 0.9898617698557086, 'subsample_freq': 6, 'min_child_samples': 27}. Best is trial 1 with value: 0.7163176253447335.
/home/bbok0525/an

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iter

/home/bbok0525/anaconda3/envs/default/lib/python3.10/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2023-07-07 13:26:02,099] Trial 9 finished with value: 0.7139987337094849 and parameters: {'max_depth': 8, 'reg_alpha': 0.6165376994399163, 'reg_lambda': 0.06472855620684219, 'num_leaves': 17, 'colsample_bytree': 0.9636162709331701, 'subsample': 0.9275674951564696, 'subsample_freq': 3, 'min_child_samples': 39}. Best is trial 1 with value: 0.7163176253447335.
/home/bbok0525/anaconda3/envs/default/lib/python3.10/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2023-07-07 13:26:02,122] Trial 10 finished with value: 0.7135582648650771 and parameters: {'max_depth': 12, 'reg_alpha': 3.4

...

[0.84781288 0.46937121 0.72517746 ... 0.4789218  0.82283308 0.46793746]
[LightGBM] [Info] Total Bins 5562
[LightGBM] [Info] Number of data: 10386, number of used features: 27
[LightGBM] [Info] Start training from score 0.665030
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[50]	training's auc: 0.722038	valid_1's auc: 0.715948
[0.80893364 0.47743953 0.72793482 ... 0.49844357 0.77498904 0.48575878]
[LightGBM] [Info] Total Bins 5562
[LightGBM] [Info] Number of data: 10386, number of used features: 27
[LightGBM] [Info] Start training from score 0.665030
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[58]	training's auc: 0.731492	valid_1's auc: 0.717121
[0.82613581 0.48599008 0.74083222 ... 0.47151998 0.80183124 0.48363837]
[LightGBM] [Info] Total Bins 5562
[LightGBM] [Info] Number of data: 10386, number of used features: 27
[LightGBM] [Info] Start training from score 0.665030
Training until v

/home/bbok0525/anaconda3/envs/default/lib/python3.10/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2023-07-07 13:26:05,970] Trial 90 finished with value: 0.7163736599631044 and parameters: {'max_depth': 4, 'reg_alpha': 2.5902463311319516e-05, 'reg_lambda': 0.0010053451167628832, 'num_leaves': 10, 'colsample_bytree': 0.9704362284562095, 'subsample': 0.8599884816165628, 'subsample_freq': 2, 'min_child_samples': 51}. Best is trial 54 with value: 0.7183035243708105.
/home/bbok0525/anaconda3/envs/default/lib/python3.10/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2023-07-07 13:26:05,995] Trial 91 finished with value: 0.7135540163386842 and parameters: {'max_depth': 4, 'reg_alph

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[67]	training's auc: 0.733746	valid_1's auc: 0.716374
[0.84182928 0.44766046 0.70413278 ... 0.49940353 0.82025168 0.46984752]
[LightGBM] [Info] Total Bins 5562
[LightGBM] [Info] Number of data: 10386, number of used features: 27
[LightGBM] [Info] Start training from score 0.665030
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[14]	training's auc: 0.709916	valid_1's auc: 0.713554
[0.74096917 0.55935116 0.72723647 ... 0.56982209 0.73493809 0.56555349]
[LightGBM] [Info] Total Bins 5562
[LightGBM] [Info] Number of data: 10386, number of used features: 27
[LightGBM] [Info] Start training from score 0.665030
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[44]	training's auc: 0.719321	valid_1's auc: 0.717019
[0.80

/home/bbok0525/anaconda3/envs/default/lib/python3.10/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2023-07-07 13:26:06,182] Trial 95 finished with value: 0.7155455417916059 and parameters: {'max_depth': 5, 'reg_alpha': 0.0013957363621028803, 'reg_lambda': 0.054395101000858, 'num_leaves': 10, 'colsample_bytree': 0.9616105609580528, 'subsample': 0.8189074168526016, 'subsample_freq': 2, 'min_child_samples': 60}. Best is trial 54 with value: 0.7183035243708105.
/home/bbok0525/anaconda3/envs/default/lib/python3.10/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
[I 2023-07-07 13:26:06,231] Trial 96 finished with value: 0.7163261223975192 and parameters: {'max_depth': 6, 'reg_alpha': 0

Early stopping, best iteration is:
[67]	training's auc: 0.734506	valid_1's auc: 0.715546
[0.83428523 0.49349389 0.70776444 ... 0.4573396  0.79484549 0.47812764]
[LightGBM] [Info] Total Bins 5562
[LightGBM] [Info] Number of data: 10386, number of used features: 27
[LightGBM] [Info] Start training from score 0.665030
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[44]	training's auc: 0.721624	valid_1's auc: 0.716326
[0.80331263 0.49848685 0.72259465 ... 0.50517502 0.78364308 0.49699655]
[LightGBM] [Info] Total Bins 5562
[LightGBM] [Info] Number of data: 10386, number of used features: 27
[LightGBM] [Info] Start training from score 0.665030
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[47]	training's auc: 0.720027	valid_1's auc: 0.71464
[0.8064906  0.48326454 0.72238137 ... 0.51315311 0.80101358 0.48545206]
[LightGBM] [Info] Total Bins 5562
[LightGBM] [Info] Number of data: 10386, number of

In [166]:
# 하이퍼 파라미터
# param1={"num_leaves":150,
#         "max_bin":200,
#        "feature_fraction":0.52,
#        "bagging_fraction":0.52,
#        "objective":"binary",
#        "learning_rate":0.05,
#        "boosting_type":"gbdt",
#        "metric":"auc"
#        }
# param2={"num_leaves":100,
#        "max_bin":200,
#        "feature_fraction":0.52,
#        "bagging_fraction":0.52,
#        "objective":"binary",
#        "learning_rate":0.05,
       
#        "metric":"auc"
#        }

param1 = {'max_depth': 5, 
          'reg_alpha': 3.0946424850432625e-06, 
          'reg_lambda': 4.1015307348324576e-06, 
          'num_leaves': 8, 
          'colsample_bytree': 0.6961655515441743,
          'subsample': 0.8632170308194024, 
          'subsample_freq': 5,
          'min_child_samples': 49,
          "metric":"auc",
          "boosting_type":"gbdt",
          }
param2 = {'max_depth': 6, 
          'reg_alpha': 1.6114208357018784e-08,
          'reg_lambda': 0.00013461567148605243,
          'num_leaves': 32,
          'colsample_bytree': 0.7544537836898809,
          'subsample': 0.8536948315468904,
          'subsample_freq': 6,
          'min_child_samples': 49,
          "metric":"auc",
          "boosting_type":"gbdt",
          }
param3 = {'max_depth': 4,
          'reg_alpha': 0.0056688664725702215,
          'reg_lambda': 1.0001169416586301e-07,
          'num_leaves': 52,
          'colsample_bytree': 0.7643558367967738,
          'subsample': 0.903085447011872,
          'subsample_freq': 7,
          'min_child_samples': 48,
          "metric":"auc",
          "boosting_type":"gbdt",
          }
param4 = {'max_depth': 4,
          'reg_alpha': 1.1368236970200307e-05, 
          'reg_lambda': 4.6208018285505224e-06, 
          'num_leaves': 8,
          'colsample_bytree': 0.9476860631912927, 
          'subsample': 0.8466146128214767,
          'subsample_freq': 2,
          'min_child_samples': 48,
          "metric":"auc",
          "boosting_type":"gbdt",
          }


params = [param4, param2]

In [167]:
# 학습

models = []
for  i in range(2):
    print(i)
    train = lgbm.Dataset(x_train, y_train.iloc[:,i])
    test = lgbm.Dataset(x_test, y_test.iloc[:,i])
    model = lgbm.train(params=params[i], train_set=train, valid_sets=[train, test], num_boost_round=500, early_stopping_rounds=10, verbose_eval=100)
    models.append(model)

0
[LightGBM] [Info] Total Bins 5562
[LightGBM] [Info] Number of data: 10386, number of used features: 27
[LightGBM] [Info] Start training from score 0.665030
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[21]	training's auc: 0.718078	valid_1's auc: 0.71661
1
[LightGBM] [Info] Total Bins 5562
[LightGBM] [Info] Number of data: 10386, number of used features: 27
[LightGBM] [Info] Start training from score 0.801464
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

In [168]:
submission = submission[['id']]

ec1 = pd.DataFrame(models[0].predict(test_df), columns=['EC1'])
ec2 = pd.DataFrame(models[1].predict(test_df), columns=['EC2'])
submission = pd.concat([submission, ec1, ec2], axis=1)

In [171]:

submission.to_csv('submit_04_3.csv', index=False)